# 工具和路由 Tools and Routing

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、通过Langchain定义工具](#二、通过Langchain定义工具)
     - [2.1 通过装饰器之间定义Tool](#2.1-通过装饰器之间定义Tool)
     - [2.2 通过pydantic类定义Tool](#2.2-通过pydantic类定义Tool)
     - [2.3 天气查询应用案例](#2.3-天气查询应用案例)
     - [2.4 通过tool定义function](#2.4-通过tool定义function)
     - [2.5 通过API定义function案例](#2.5-通过API定义function案例)
 - [三、 路由](#三、路由)
     - [3.1 Tool转换为Function](#3.1-Tool转换为Function)
     - [3.2 通过route进行tools的选择](#3.2-通过route进行tools的选择)
     - [3.3 输出解析器](#3.3-输出解析器)
 - [四、总结](#四、总结)


# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

# 二、通过Langchain定义工具

## 2.1 通过装饰器之间定义Tool

- 中文Prompt

In [ ]:
# tool装饰器包装了search函数
@tool
def search_zh(query: str) -> str:
    """在网络上查询天气"""
    return "42度"

In [ ]:
# 搜索工具的函数名
print(search.name)
#搜索工具的功能描述（即函数注释）
print(search.description)
# 搜索工具需要传递的参数
print(search.args)

search
search(query: str) -> str - "Search for weather online
{'query': {'title': 'Query', 'type': 'string'}}


## 2.2 通过 pydantic 类定义 Tool

In [ ]:
# 导入 Pydantic 库中的 BaseModel 类和 Field 函数，它们用于定义数据模型和字段。
from pydantic import BaseModel, Field

class SearchInput_zh(BaseModel):
    """
    定义了 SearchInput 类中的一个属性 query，它是一个字符串类型。通过 Field 函数，你为这个字段提供了一些配置
    其中 description 参数用于描述这个字段的用途，即 "Thing to search for"（要搜索的内容）。
    """
    query: str = Field(description="你需要搜索的东西")


In [ ]:
# args_schema参数传递SearchInput工具类
@tool(args_schema=SearchInput_zh)
def search_zh(query: str) -> str:
    """在网上查找温度"""
    return "42度"

In [ ]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [ ]:
search.run("圣弗朗西斯科")

'42f'

## 2.3 天气查询应用案例
整体代码逻辑：
1. 使用 Pydantic 定义了输入类OpenMeteoInput，以及输入的两个参数（经度和纬度）的输入格式
2. 定义了一个函数 get_current_temperature，该函数使用 OpenMeteo API 获取给定坐标位置的当前温度。
3. get_current_temperature函数通过发送 HTTP 请求获取 API 响应，然后从响应中提取并计算出当前时间对应的温度。
4. get_current_temperature函数返回一个字符串，其中包含了当前温度的信息。

In [ ]:
# 导入所需的库
import requests
from pydantic import BaseModel, Field
import datetime

# 定义输入类（input schema）
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for") #要获取天气数据的位置的纬度
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for") #要获取天气数据的位置的经度

# 使用 @tool 装饰器并指定输入模型
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"Fetch current temperature for given coordinates."""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.utcnow()
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'The current temperature is {current_temperature}°C'


In [ ]:
# 工具的名字
print(get_current_temperature.name)
# 工具的功能描述
print(get_current_temperature.description)
# 工具的输入参数
print(get_current_temperature.args)

get_current_temperature
get_current_temperature(latitude: float, longitude: float) -> dict - "Fetch current temperature for given coordinates.
{'latitude': {'title': 'Latitude', 'description': 'Latitude of the location to fetch weather data for', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}}


## 2.4-通过tool定义function

In [ ]:
# 导入openai的模板
from langchain.tools.render import format_tool_to_openai_function

In [ ]:
# 将定义好的工具直接传入模板，打印tool的名字、描述和输入参数格式
format_tool_to_openai_function(get_current_temperature)

C:\Users\14549\anaconda3\envs\basic\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - "Fetch current temperature for given coordinates.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [ ]:
# 调用工具
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 23.9°C'

In [ ]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
# 工具的名字
search_wikipedia.name

'search_wikipedia'

In [ ]:
# 工具的描述
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [ ]:
# 将工具格式化为 OpenAI 函数
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [ ]:
# 调用
search_wikipedia({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI\nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the AI spring.The organization consists of the non-profit OpenAI, Inc. re

## 2.5 通过API定义function案例

In [ ]:
# openapi_spec_to_openai_fn可以把json格式的API定义转换成openai的function call格式
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# OpenAPISpec是标准化的API格式定义
from langchain.utilities.openapi import OpenAPISpec

In [ ]:
# json格式的API定义
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [ ]:
# 从text中导入API的详细定义
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [ ]:
# 转换成openai的fuction call格式
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [ ]:
# 查看fuction的定义
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [ ]:
# 导入模型
from langchain.chat_models import ChatOpenAI

In [ ]:
# 设置模型温度系数并传入function
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

C:\Users\14549\anaconda3\envs\basic\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
# 输入query，查看模型调用的function以及返回信息
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}})

In [ ]:
model.invoke("这三只宠物的名字叫什么？")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}})

In [ ]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}})

In [ ]:
model.invoke("告诉我id为42的宠物的消息")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}})

# 三、路由

展示一个函数调用的例子，用于在两个候选函数之间做出决策。



## 3.1 Tool转换为Function

鉴于我们上面提到的工具，让我们将它们格式化为 OpenAI 函数，并展示相同的行为。

In [ ]:
# 将工具格式化为 OpenAI 函数
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [ ]:
# 模型调用
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}})

In [ ]:
# 模型调用
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}})

In [ ]:
# 使用template构造prompt
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

prompt_zh = ChatPromptTemplate.from_messages([
    ("system", "你是个乐于助人的助理"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt和model连接起来
chain = prompt | model

In [ ]:
# 输入query进行调用
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}})

In [ ]:
# 导入输出解析的包
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
# 创建处理链，将 prompt、model 和 OpenAIFunctionsAgentOutputParser 连接起来
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
# 调用
result = chain.invoke({"input": "what is the weather in sf right now"})

In [ ]:
# 打印返回的类型，可以判断是否产生function的调用
type(result)

langchain_core.agents.AgentActionMessageLog

In [ ]:
# 查看调用的tool
result.tool

'get_current_temperature'

In [ ]:
# 查看tool的输入，result.message_log可以查看调用结果
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [ ]:
# 调用的获取温度的工具
get_current_temperature(result.tool_input)

'The current temperature is 9.4°C'

In [ ]:
# 继续调用
result = chain.invoke({"input": "hi!"})

In [ ]:
# 打印返回的类型，可以判断是否产生function的调用
type(result)

langchain_core.agents.AgentFinish

In [ ]:
# 查看返回值
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

## 3.2-通过route进行tools的选择

In [ ]:
"""
route会根据result进行tools的选择：
AgentFinish：表示已经完成，可以输出
AgentActionMessageLog：表示未完成，需要继续进行route调用tools
"""
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
# 创建处理链，将 prompt、model 、OpenAIFunctionsAgentOutputParser和route 连接起来
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
chain_zh = prompt_zh | model | OpenAIFunctionsAgentOutputParser() | route

In [ ]:
# 调用
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}})

In [ ]:
result_zh = chain_zh.invoke({"input": "圣弗朗西斯科的天气现在怎么样？"})
result_zh

'The current temperature is 9.4°C'

In [ ]:
# 调用
result = chain.invoke({"input": "What is langchain?"})
result

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI\nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the AI spring.The organization consists of the non-profit OpenAI, Inc. re

In [ ]:
result_zh = chain_zh.invoke({"input": "什么是langchain?"})
result_zh

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI\nSummary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the AI spring.The organization consists of the non-profit OpenAI, Inc. re

In [ ]:
# 调用并查看结果
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'

In [ ]:
chain_zh.invoke({"input": "你好！"})

'你好！有什么可以帮助你的吗？'

# 四、总结

我们总结一下完整的调用流程：

构造Prompt --> 调用模型 --> 解析模型返回的结果 --> 进行路由选择对应的tool


# 五、英文版提示

## 2.1 通过装饰器之间定义Tool

- 英文Prompt

In [ ]:
# 导入tool包
from langchain.agents import tool

In [ ]:
# tool装饰器包装了search函数
@tool
def search(query: str) -> str:
    """"Search for weather online"""
    return "42f"

In [ ]:
# 搜索工具的函数名
print(search.name)
#搜索工具的功能描述（即函数注释）
print(search.description)
# 搜索工具需要传递的参数
print(search.args)

search
search(query: str) -> str - "Search for weather online
{'query': {'title': 'Query', 'type': 'string'}}


## 2.2 通过 pydantic 类定义 Tool

In [ ]:
# 导入 Pydantic 库中的 BaseModel 类和 Field 函数，它们用于定义数据模型和字段。
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    """
    定义了 SearchInput 类中的一个属性 query，它是一个字符串类型。通过 Field 函数，你为这个字段提供了一些配置
    其中 description 参数用于描述这个字段的用途，即 "Thing to search for"（要搜索的内容）。
    """
    query: str = Field(description="Thing to search for")

In [ ]:
# args_schema参数传递SearchInput工具类
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [ ]:
# 搜索工具类需要传递的参数
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [ ]:
# 启动搜索工具，query为sf
search.run("sf")

'42f'